In [30]:
class animal:
    id = 1
    def __init__(self, val:int) -> None:
        self.val = val
        # self.id = animal.id
        # animal.id += 1
    
    
class human(animal):
    def __init__(self, val) -> None:
        super().__init__(val)
        self.id = animal.id + 1
        animal.id += 1
        


In [31]:
a1 = animal(10)
print(a1.id)
h1 = human(20)
print(a1.id, h1.id)
h2 = human(30)
print(a1.id, h1.id, h2.id)

1
2 2
3 2 3


In [1]:
import heapq

class Node:
    def __init__(self, row, col, parent=None):
        self.row = row
        self.col = col
        self.parent = parent
        self.g = 0  # Cost from start node to current node
        self.h = 0  # Heuristic (estimated cost from current node to end node)
        self.f = 0  # Total cost (g + h)

def heuristic(node, goal):
    return abs(node.row - goal.row) + abs(node.col - goal.col)

def get_neighbors(node, grid):
    neighbors = []
    directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]
    
    for dr, dc in directions:
        new_row, new_col = node.row + dr, node.col + dc
        
        if 0 <= new_row < len(grid) and 0 <= new_col < len(grid[0]) and grid[new_row][new_col] != 1:
            neighbors.append(Node(new_row, new_col))
    
    return neighbors

def a_star(grid, start, goal):
    open_list = []
    closed_set = set()
    
    heapq.heappush(open_list, (start.f, start))
    
    while open_list:
        _, current_node = heapq.heappop(open_list)
        
        if (current_node.row, current_node.col) == (goal.row, goal.col):
            path = []
            while current_node:
                path.insert(0, (current_node.row, current_node.col))
                current_node = current_node.parent
            return path
        
        closed_set.add((current_node.row, current_node.col))
        
        neighbors = get_neighbors(current_node, grid)
        for neighbor in neighbors:
            if (neighbor.row, neighbor.col) in closed_set:
                continue
            
            tentative_g = current_node.g + 1
            if neighbor not in open_list or tentative_g < neighbor.g:
                neighbor.g = tentative_g
                neighbor.h = heuristic(neighbor, goal)
                neighbor.f = neighbor.g + neighbor.h
                neighbor.parent = current_node
                
                if (neighbor.row, neighbor.col) not in {(item[1].row, item[1].col) for item in open_list}:
                    heapq.heappush(open_list, (neighbor.f, neighbor))
    
    return None  # No path found

# Example usage
grid = [
    [0, 0, 0, 0, 0],
    [0, 1, 1, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 1, 1, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0]
]

start_node = Node(0, 0)
goal_node = Node(5, 4)

path = a_star(grid, start_node, goal_node)
if path:
    print("Shortest path found:", path)
else:
    print("No path found")


TypeError: '<' not supported between instances of 'Node' and 'Node'

In [1]:
from chess_info import *
from json import dumps

dumps(rabbit())

SyntaxError: invalid syntax (chess_info.py, line 1189)